In [ ]:
import pandas as pd
import numpy
import sklearn
import scipy
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from math import radians, cos, sin, asin, sqrt 
from scipy.cluster.vq import whiten
from scipy.cluster.vq import *  
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def haversine(lonlat1, lonlat2):#判断两点间的球面距离
    lat1, lon1 = lonlat1
    lat2, lon2 = lonlat2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6378
    return c * r

In [ ]:
fr3 = open('/data/locus/data_feature_1_2_2.csv','r') 
latitude = []
longitude = []
for line in fr3.readlines():
    item = [k.strip() for k in line.split(',')]
    latitude.append(float(item[1]))
    longitude.append(float(item[2]))

In [ ]:
lati = latitude[1000:4000]#由于dbscan复杂度高，所以选取其中的3000个数据来测试。
longi = longitude[1000:4000]
X = pd.DataFrame ({"lat":lati,"lng":longi})
distance_matrix = squareform(pdist(X, (lambda u, v: haversine(u, v))))
db = DBSCAN(eps=3, min_samples=20, metric='precomputed')  
y_db = db.fit_predict(distance_matrix)

X['cluster'] = y_db

results = {}
for i in X.values:
    if i[2] not in results.keys():
         results[i[2]] = [[i[1], i[0]]]
    else:
        if results[i[2]]:
            results[i[2]].append([i[1], i[0]])
        else:
            results[i[2]] = [[i[1], i[0]]]
print ("DBSCAN output:", len(results), results.keys())

In [ ]:
    fig, ax = plt.subplots()
    ax.scatter(X['lat'], X['lng'], c=X['cluster'])
    plt.show()